In [1]:
import openai
import pandas as pd
import tqdm
import glob

import warnings
warnings.filterwarnings("ignore")

In [2]:
with open("../apikey/apikey.txt", "r") as f:
    openai.api_key = f.readline().replace("\n", "")

In [3]:
def query_prompt(prompt, max_tokens=5000):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        temperature=0,
    )
    return response["choices"][0]["message"]["content"]

In [4]:
def get_df(fname):
    def f(categories):
        for cat in categories:
            for word in [
                "geology",
                "science",
                "physics",
                "chemistry",
                "mathematical",
                "biology",
                "astronomy",
                "ecology",
                "genetics",
            ]:
                if word.lower() in cat.lower():
                    return True
            
        return False
    
    def text_preprocess(text):
        return text.replace("===", "\n").replace("==", "\n")

    df = pd.read_parquet(fname)
    df_science = df[df["categories"].apply(f)]
    df_science["text"] = "title: " + df_science["title"] + "\n" + df_science["text"].apply(text_preprocess)
    return df_science.sample(len(df_science)//100)

In [5]:
files = glob.glob("../data/wikipedia/*.parquet")

In [6]:
import time

In [7]:
import pickle

In [8]:
import json

In [9]:
from datetime import datetime as dt
import os

In [10]:
texts = []

In [11]:
import traceback 
batch_size = 1

def make_prompt(series):
    prompt = f"""
# 依頼
You are a professor at a science university and are creating a test for your students.
Using the given text, create a question in which you select the most appropriate statement from the five options in the question text. Also, extract the evidence for your answer.
The output should be an array in json format, with "prompt" as the problem statement, "A," "B," "C," "D," and "E" as choices, "answer" as the answer choice (one of A through E), and "basis" as the rationale. Please make sure that the answer choices are not all the same, e.g., all five answers are A.

# text
## text 1 
title: {series['title']}

{series['text']}


# attention
Please create 7 question.
"""
    return prompt

def f(series):
    if series["A"] != series["A"]:
        if type(series["choices"]) == dict:
            for key in ["A", "B", "C", "D", "E"]:
                series[key] = series["choices"][key]
        elif type(series["choices"] == list):
            for i, key in enumerate(["A", "B", "C", "D", "E"]):
                series[key] = series["choices"][i]
    return series

now_date = dt.now().strftime("%Y%m%d%H%M%S")

first = True
for file in files:
    if os.path.basename(file) in ["all.parquet"]:
        print(f"pass: {file}")
        continue
    df_science = get_df(file)
    
    for i in tqdm.tqdm(range(len(df_science)), desc=file):
        try:
            series = df_science.iloc[i]
            prompt = make_prompt(series)
            text = query_prompt(prompt)
            texts_json = json.loads(text)
            for text_json in texts_json:
                text_json["wiki_id"] = series["id"]
                text_json["original_text"] = series["text"]
                texts.append(text_json)
            if first:
                print(texts_json)
                first = False
        except Exception as e:
            print(e)
            traceback.print_exc()
            print(text)
        if i % 20 == 0:
            df_texts = pd.DataFrame(texts)
            df_texts = df_texts.apply(f, axis=1)

            df_texts.to_csv(f"output_gpt3.5_generate/{now_date}.csv")

../data/wikipedia/a.parquet:   1%|██▌                                                                                                                                                                                                               | 1/83 [00:29<40:18, 29.49s/it]

[{'prompt': "What is Allison Arieff's current position?", 'A': 'Editorial Director of Print for the MIT Technology Review', 'B': 'Senior Editor of City Monitor - The New Statesman', 'C': 'Editorial Director for SPUR', 'D': 'Editor-in-chief of Dwell magazine', 'E': 'Senior content lead for IDEO', 'answer': 'A', 'basis': 'The text states that Allison Arieff is the Editorial Director of Print for the MIT Technology Review.', 'wiki_id': '4378675', 'original_text': 'title: Allison Arieff\nAllison Arieff (born October 29, 1966Adam Bluestein, So What Do You Do, Allison Arieff? (Interview), mediabistro, July 29, 2003. Retrieved 2012-03-05.) is the Editorial Director of Print for the MIT Technology Review. An American writer and editor, she was, most recently, Senior Editor of City Monitor - The New Statesman. She was previously editorial director for the urban planning and policy think tank, SPUR (the San Francisco Bay Area Planning and Urban Research Association). From 2007-2020, she wrote a 

../data/wikipedia/a.parquet:  16%|████████████████████████████████▋                                                                                                                                                                                | 13/83 [06:12<34:35, 29.65s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_2063/1584694044.py", line 47, in <module>
    texts_json = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 353, in raw_decode
    obj, end = self.scan_once(s, idx)
json.decoder.JSONDecodeError: Expecting ',' delimiter: line 23 column 42 (char 1089)
../data/wikipedia/a.parquet:  17%|███████████████████████████████████▎                                                                                      

Expecting ',' delimiter: line 23 column 42 (char 1089)
[
  {
    "prompt": "What is an antibody microarray?",
    "A": "A technology used for detecting protein expression from various biofluids",
    "B": "A technology used for detecting DNA sequences",
    "C": "A technology used for detecting chemical compounds",
    "D": "A technology used for detecting tissue samples",
    "E": "A technology used for detecting protein phosphorylation",
    "answer": "A",
    "basis": "In the given text, it is mentioned that an antibody microarray is a specific form of protein microarray used for detecting protein expression from various biofluids including serum, plasma, and cell or tissue lysates."
  },
  {
    "prompt": "Who introduced the concept and methodology of antibody microarrays?",
    "A": "Roger Ekins",
    "B": "Tse Wen Chang",
    "C": "Nancy T. Chang",
    "D": "Ruo-Pan Huang",
    "E": "RayBiotech, Inc.",
    "answer": "Tse Wen Chang",
    "basis": "In the given text, it is mentione

../data/wikipedia/a.parquet:  51%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                                                       | 42/83 [21:02<20:26, 29.92s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_2063/1584694044.py", line 47, in <module>
    texts_json = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 340, in decode
    raise JSONDecodeError("Extra data", s, end)
json.decoder.JSONDecodeError: Extra data: line 2 column 1 (char 444)
../data/wikipedia/a.parquet:  52%|████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                    | 43/83 [21:34<20:31, 30.78s/it]

...

../data/wikipedia/w.parquet:  54%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                                                                 | 15/28 [05:13<04:28, 20.64s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_2063/1584694044.py", line 47, in <module>
    texts_json = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 353, in raw_decode
    obj, end = self.scan_once(s, idx)
json.decoder.JSONDecodeError: Expecting ',' delimiter: line 63 column 49 (char 3248)
../data/wikipedia/w.parquet:  57%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍  

Expecting ',' delimiter: line 63 column 49 (char 3248)
[
  {
    "prompt": "What is the main theme of the novel Wild Jack?",
    "A": "The struggle for survival in a post-apocalyptic world",
    "B": "The importance of friendship and loyalty",
    "C": "The corrupting influence of power",
    "D": "The clash between civilization and the wilderness",
    "E": "The consequences of questioning societal norms",
    "answer": "D",
    "basis": "The text mentions that the novel explores the dichotomy between the city and the wild, with the protagonist experiencing true freedom in the wilderness. This suggests that the main theme of the novel is the clash between civilization and the wilderness."
  },
  {
    "prompt": "What is the reason for Clive's imprisonment on the prison island?",
    "A": "He is falsely accused of questioning the status quo",
    "B": "He is caught listening in on a conversation",
    "C": "He is accused of organizing a rebellion against the authorities",
    "D": "He 

../data/wikipedia/x.parquet:  50%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                                         | 1/2 [00:19<00:19, 19.39s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_2063/1584694044.py", line 47, in <module>
    texts_json = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 353, in raw_decode
    obj, end = self.scan_once(s, idx)
json.decoder.JSONDecodeError: Expecting ',' delimiter: line 53 column 29 (char 2351)
../data/wikipedia/x.parquet: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

Expecting ',' delimiter: line 53 column 29 (char 2351)
[
  {
    "prompt": "What is the title of the third book in the Ender's Game series?",
    "A": "Xenocide",
    "B": "Speaker for the Dead",
    "C": "Children of the Mind",
    "D": "Ender's Game",
    "E": "Gloriously Bright",
    "answer": "A",
    "basis": "The text states that Xenocide is the third book in the Ender's Game series."
  },
  {
    "prompt": "What is the meaning of the title 'Xenocide'?",
    "A": "The act of killing populations of aliens",
    "B": "The act of killing humans infected with the descolada virus",
    "C": "The act of destroying the entire planet of Lusitania",
    "D": "The act of killing the pequeninos",
    "E": "The act of killing the Hive Queen",
    "answer": "A",
    "basis": "The text explains that the title 'Xenocide' is a combination of 'xeno-', meaning alien, and '-cide', referring to the act of killing, together meaning the act of killing populations of aliens."
  },
  {
    "prompt": "Wh


../data/wikipedia/z.parquet: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:29<00:00, 17.86s/it]


In [14]:
df_texts.to_csv(f"output_gpt3.5_generate/{now_date}.csv")

In [23]:
for i in range(10):
    series = df_texts.sample(1).iloc[0]
    print(f"{series['prompt']} \n text: \n {series['original_text'].split('==')[0]}")
    print("----------------")

What is the maximum sustainable yield (MSY) usually higher than? 
 text: 
 title: Maximum sustainable yield
In population ecology and economics, maximum sustainable yield (MSY) is theoretically, the largest yield (or catch) that can be taken from a species' stock over an indefinite period. Fundamental to the notion of sustainable harvest, the concept of MSY aims to maintain the population size at the point of maximum growth rate by harvesting the individuals that would normally be added to the population, allowing the population to continue to be productive indefinitely. Under the assumption of logistic growth, resource limitation does not constrain individuals' reproductive rates when populations are small, but because there are few individuals, the overall yield is small. At intermediate population densities, also represented by half the carrying capacity, individuals are able to breed to their maximum rate. At this point, called the maximum sustainable yield, there is a surplus of i

In [40]:
df_texts.to_csv(f"output_gpt3.5_generate/{now_date}.csv")

In [20]:
pd.DataFrame(texts)["A"].isnull().sum()

69

In [41]:
df_science = get_df(files[0])
df_science[df_science["id"] == '29637793']

,id,title,text,categories


In [42]:
df_science

,id,title,text,categories
368626,1908395,Artificial brain,title: Artificial brain\nAn artificial brain (...,"[Computational neuroscience, Robotics, Emergin..."
34268,10410698,Abdul Amir al-Jamri,title: Abdul Amir al-Jamri\nSheikh Abdul Amir ...,"[1938 births, 2006 deaths, Deaths from kidney ..."
233077,1958222,Amyloid beta,title: Amyloid beta\nAmyloid beta (Aβ or Abeta...,"[Peptides, Molecular neuroscience, Alzheimer's..."
9870,3621668,A Woman of the Iron People,title: A Woman of the Iron People\nA Woman of ...,"[1991 American novels, 1991 science fiction no..."
139719,38366604,Albert Spaier,title: Albert Spaier\nAlbert Spaier (9 July 18...,"[1883 births, 1934 deaths, Writers from Iași, ..."
...,...,...,...,...
62685,4474244,Actuarial reserves,"title: Actuarial reserves\nIn insurance, an ac...","[Actuarial science, Capital requirement de:Dec..."
357456,4260564,Arrival II,title: Arrival II\nArrival ll (alternatively t...,"[1998 films, 1998 science fiction films, Ameri..."
391514,32894329,Astrobiophysics,title: Astrobiophysics\nAstrobiophysics is a f...,"[Astrophysics, Biophysics .]"
10000,2824171,A World of Difference (novel),title: A World of Difference (novel)\nA World ...,"[1990 American novels, Novels set during the C..."
